# [CityBikes](https://citybik.es/)

### Send a request to CityBikes for the city of your choice. 

In [7]:
import requests
import json
import pandas as pd

In [8]:
# Call fields, name, href, and location to figure out city endpoint
url = 'http://api.citybik.es/v2/networks'
response = requests.get(url)
print(response)
networks = response.json()

<Response [200]>


In [9]:
# Finding index of Redding, CA in networks
for i in range(len(networks['networks'])):
    if 'Redding' in networks['networks'][i]['location']['city']:
        index = i

print(index)
print(networks['networks'][index]['href'])


688
/v2/networks/redding


In [10]:
href = networks['networks'][index]['href']

In [11]:
# Sending request for Redding bike stations with Href endpoint
url = fr'http://api.citybik.es{href}'
response = requests.get(url)
results = response.json()

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

#### Keys from Results Json Dictionary

results.keys() <br>
dict_keys(['networks'])

results['network'].keys() <br>
dict_keys(['company', 'gbfs_href', 'href', 'id', 'location', 'name', 'stations'])

results['network']['location'].keys() <br>
dict_keys(['city', 'country', 'latitude', 'longitude'])

results['network']['stations'][0].keys() <br>
dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])



In [12]:
print(results['network']['company'])
print(results['network']['gbfs_href'])
print(results['network']['href'])
print(results['network']['id'])
print(results['network']['location'])
print(results['network']['name'])

['BCycle, LLC']
https://gbfs.bcycle.com/bcycle_redding/gbfs.json
/v2/networks/redding
redding
{'city': 'Redding, CA', 'country': 'US', 'latitude': 40.5875, 'longitude': -122.386}
Redding BCycle


### Put your parsed results into a DataFrame.

In [13]:
ReddingStations = pd.json_normalize(results['network']['stations'])

In [21]:
ReddingStations.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.uid
0,2,2,bb7665847eda4d3b5654ef23584bba2f,40.58002,-122.38890,Market and Gold Street,2024-05-25T18:15:37.982000Z,Market Street & Gold Street,2,True,1716660937,0,1,1,bcycle_redding_7427
1,3,2,3bb59ffd3799adff3f475e8d3985532f,40.58352,-122.39042,Market Center South,2024-05-25T18:15:38.034000Z,Market Street & Yuba Street,2,True,1716660937,0,1,1,bcycle_redding_7426
2,2,3,08aae6d19bdfffb93561437caf892f36,40.58435,-122.39067,Market Center North,2024-05-25T18:15:38.035000Z,1371 Butte Street,3,True,1716660937,0,1,1,bcycle_redding_7425
3,3,0,dd5c0ef4079f9e920b3cdba5c3ca167c,40.59502,-122.40150,Diestelhorst Bridge,2024-05-25T18:15:38.036000Z,Benton Drive,0,True,1716660937,0,1,1,bcycle_redding_7424
4,3,3,64a7ac84a24f102734b87abc4ba71af0,40.57337,-122.38177,Redding City Hall,2024-05-25T18:15:38.037000Z,777 Cypress Ave,3,True,1716660937,0,1,1,bcycle_redding_7423


In [18]:
# Dropping uneccessary url columns
ReddingStations.drop(columns=['extra.rental_uris.android','extra.rental_uris.ios'],inplace=True)

In [43]:
# Renaming columns to be descriptive and clear to what they are
ReddingStations.rename(columns={'extra.address': 'address', 'extra.ebikes': 'free_ebikes','extra.normal_bikes': 'free_normal_bikes','extra.renting':'renting',
                                'extra.returning':'returning','extra.uid':'uid','extra.last_updated': 'last_updated','extra.has_ebikes':'has_ebikes'}, inplace=True
                                )

In [44]:
# Adding total bike slots column for each station 
ReddingStations['total_slots'] = ReddingStations['empty_slots'] + ReddingStations['free_bikes']

In [45]:
# Existing Columns in df
ReddingStations.columns.values

array(['id', 'uid', 'empty_slots', 'free_bikes', 'total_slots',
       'latitude', 'longitude', 'name', 'address', 'free_ebikes',
       'has_ebikes', 'free_normal_bikes', 'timestamp', 'last_updated',
       'renting', 'returning'], dtype=object)

In [48]:
# Rearranging column order
cols = ['id', 'uid', 'empty_slots', 'free_bikes', 'total_slots', 'latitude', 'longitude', 'name',
        'address', 'free_ebikes', 'has_ebikes', 'free_normal_bikes', 'timestamp', 'last_updated', 'renting', 'returning'
       ]
ReddingStations = ReddingStations[cols]

In [50]:
# Saving dataframe to csv. Commented out as to not accidentally overwrite existing files
# ReddingStations.to_csv('../data/ReddingBikesDF.csv',index=False)